In [1]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import djvu.decode

In [5]:
print(djvu.decode.DDJVU_VERSION)

26


In [8]:
context = djvu.decode.Context()
document = context.new_document(djvu.decode.FileUri('data/hayeren.djvu'))
document.decoding_job.wait()

In [100]:
pages = []

for page in document.pages:
    val = page.text.sexpr.value

    line_delims_and_words = [y for x in val if type(x) == tuple for y in x if type(y) != int]

    lines = []
    current_line = []

    for entry in line_delims_and_words[1:]:
        if type(entry) == tuple:
            current_line.append(entry[-1])
        else:
            lines.append(' '.join(current_line))
            current_line = []
    lines.append(' '.join(current_line))

    pages.append(lines)
    
core_lines = [line.strip() for page in pages[16:-1] for line in page[:-2]]

# get section headers from pages 4-11, ignore page numbers in page headers
section_headers = [line for 
    page in pages[4:12] for line in page[1:-1] if any(not c in '0123456789' for c in line)][3:]

section_headers

['ОСНОВНЫЕ ПОНЯТИЯ',
 'основные понятия | часть 1',
 '1. Местоимения',
 '2. Приветствия. Прощания. Извинения. Благодарность',
 '3. Обращения',
 '4. Числаот1до100',
 '5. Числа от 100',
 '6. Числа. Порядковые числительные',
 '7. Числа. Дроби',
 '8. Числа. Математические действия',
 '9. Числа. Разное',
 '10. Самые важные глаголы - 1',
 '11. Самые важные глаголы - 2',
 '12. Самые важные глаголы - 3',
 '13. Самые важные глаголы - 4',
 '14. Цвета',
 '15. Вопросы',
 '16. Основные предлоги',
 '17. Вводные и служебные слова. Наречия - 1',
 '18. Вводные и служебные слова. Наречия - 2',
 'основные понятия | часть 2',
 '19. Дни недели',
 '20. Часы. Время суток',
 '21. Месяцы. Времена года',
 '22. Время. Разное',
 '23. Противоположности',
 '24. Линии и формы',
 '25. Меры измерения',
 '26. Ёмкости',
 '27. Материалы',
 '28. Металлы',
 'ЧЕЛОВЕК',
 'человек | тело человека',
 '29. Человек. Общие понятия',
 '30. Анатомия',
 '31. Голова',
 '32. Тело',
 'одежда | аксессуары',
 '33. Верхняя одежда',
 '34. 

In [102]:
# check that all section_headers do actually appear in text
for sh in section_headers:
    print('{} {}'.format('✅' if sh in core_lines else '❌', sh))

✅ ОСНОВНЫЕ ПОНЯТИЯ
✅ основные понятия | часть 1
✅ 1. Местоимения
✅ 2. Приветствия. Прощания. Извинения. Благодарность
✅ 3. Обращения
✅ 4. Числаот1до100
✅ 5. Числа от 100
✅ 6. Числа. Порядковые числительные
✅ 7. Числа. Дроби
✅ 8. Числа. Математические действия
✅ 9. Числа. Разное
✅ 10. Самые важные глаголы - 1
✅ 11. Самые важные глаголы - 2
✅ 12. Самые важные глаголы - 3
✅ 13. Самые важные глаголы - 4
✅ 14. Цвета
✅ 15. Вопросы
✅ 16. Основные предлоги
✅ 17. Вводные и служебные слова. Наречия - 1
✅ 18. Вводные и служебные слова. Наречия - 2
✅ основные понятия | часть 2
✅ 19. Дни недели
✅ 20. Часы. Время суток
✅ 21. Месяцы. Времена года
✅ 22. Время. Разное
✅ 23. Противоположности
✅ 24. Линии и формы
✅ 25. Меры измерения
✅ 26. Ёмкости
✅ 27. Материалы
✅ 28. Металлы
✅ ЧЕЛОВЕК
✅ человек | тело человека
✅ 29. Человек. Общие понятия
✅ 30. Анатомия
✅ 31. Голова
✅ 32. Тело
✅ одежда | аксессуары
✅ 33. Верхняя одежда
✅ 34. Одежда
✅ 35. Одежда. Бельё
✅ 36. Головной убор
✅ 37. Обувь
✅ 38. Ткани. Матери

In [99]:
# get section headers from pages 4-11, ignore page numbers in page headers
section_headers = [line for 
    page in pages[4:12] for line in page[1:-1] if any(not c in '0123456789' for c in line)][3:]

['ОСНОВНЫЕ ПОНЯТИЯ',
 'основные понятия | часть 1',
 '1. Местоимения',
 '2. Приветствия. Прощания. Извинения. Благодарность',
 '3. Обращения',
 '4. Числаот1до100',
 '5. Числа от 100',
 '6. Числа. Порядковые числительные',
 '7. Числа. Дроби',
 '8. Числа. Математические действия',
 '9. Числа. Разное',
 '10. Самые важные глаголы - 1',
 '11. Самые важные глаголы - 2',
 '12. Самые важные глаголы - 3',
 '13. Самые важные глаголы - 4',
 '14. Цвета',
 '15. Вопросы',
 '16. Основные предлоги',
 '17. Вводные и служебные слова. Наречия - 1',
 '18. Вводные и служебные слова. Наречия - 2',
 'основные понятия | часть 2',
 '19. Дни недели',
 '20. Часы. Время суток',
 '21. Месяцы. Времена года',
 '22. Время. Разное',
 '23. Противоположности',
 '24. Линии и формы',
 '25. Меры измерения',
 '26. Ёмкости',
 '27. Материалы',
 '28. Металлы',
 'ЧЕЛОВЕК',
 'человек | тело человека',
 '29. Человек. Общие понятия',
 '30. Анатомия',
 '31. Голова',
 '32. Тело',
 'одежда | аксессуары',
 '33. Верхняя одежда',
 '34. 

In [62]:
unicodeblock.blocks.of('б')

'CYRILLIC'

In [67]:
pages[16]

['ОСНОВНЫЕ ПОНЯТИЯ',
 'основные понятия | часть 1',
 '1. Местоимения',
 'я',
 'եես',
 'ты',
 'դու',
 'он, она, оно',
 'նա',
 'мы',
 'մենք',
 'вы',
 'դուք',
 'они',
 'նրանք',
 '2. Приветствия. Прощания. Извинения. Благодарность',
 'Здравствуй!',
 'Բարեւ',
 'Здравствуйте!',
 'Բարեւ՜ ձեզ',
 'Доброе утро!',
 'Բարի լո՜ւյս',
 'Добрый день!',
 'Բարի օ՜ր',
 'Добрый вечер!',
 'Բարի երեկո՜',
 'здороваться',
 'բարեւել',
 'Привет!',
 'Ողջո՜ւյն',
 'привет, приветствие',
 'ողջույն',
 'приветствовать',
 'ողջունել',
 'Как дела?',
 'Ո՞նց են գործերդ',
 'Что нового?',
 'Ի՞նչ նորություն',
 'До свидания!',
 'Ցտեսությո՜ւն',
 'До скорой встречи!',
 'Մինչ նոր հանդիպո՜ւմ',
 'Прощай! Прощайте!',
 'Մնաս բարո՜վ',
 'прощаться',
 'հրաժեշտ տալ',
 'Пока!',
 'Առա՜յժմ',
 'Спасибо!',
 'Շնորհակալությո՜ւն',
 'Большое спасибо!',
 'Շատ շնորհակա՜լ եմ',
 'Пожалуйста (ответ)',
 'Խնդրեմ',
 'Не стоит благодарности',
 'Հոգ չէ',
 'Не за что',
 'չարժե',
 'Извини!',
 'Ներողությո՜ւն',
 'Извините!',
 'Ներեցե՜ք',
 'извинять',
 'ներել',

In [69]:
[unicodeblock.blocks.of(c) for c in 'կա՜լ']

['ARMENIAN', 'ARMENIAN', 'ARMENIAN', 'ARMENIAN']

In [79]:
[line.strip() for page in pages[16:-1] for line in page[:-2]]

['ОСНОВНЫЕ ПОНЯТИЯ',
 'основные понятия | часть 1',
 '1. Местоимения',
 'я',
 'եես',
 'ты',
 'դու',
 'он, она, оно',
 'նա',
 'мы',
 'մենք',
 'вы',
 'դուք',
 'они',
 'նրանք',
 '2. Приветствия. Прощания. Извинения. Благодарность',
 'Здравствуй!',
 'Բարեւ',
 'Здравствуйте!',
 'Բարեւ՜ ձեզ',
 'Доброе утро!',
 'Բարի լո՜ւյս',
 'Добрый день!',
 'Բարի օ՜ր',
 'Добрый вечер!',
 'Բարի երեկո՜',
 'здороваться',
 'բարեւել',
 'Привет!',
 'Ողջո՜ւյն',
 'привет, приветствие',
 'ողջույն',
 'приветствовать',
 'ողջունել',
 'Как дела?',
 'Ո՞նց են գործերդ',
 'Что нового?',
 'Ի՞նչ նորություն',
 'До свидания!',
 'Ցտեսությո՜ւն',
 'До скорой встречи!',
 'Մինչ նոր հանդիպո՜ւմ',
 'Прощай! Прощайте!',
 'Մնաս բարո՜վ',
 'прощаться',
 'հրաժեշտ տալ',
 'Пока!',
 'Առա՜յժմ',
 'Спасибо!',
 'Շնորհակալությո՜ւն',
 'Большое спасибо!',
 'Շատ շնորհակա՜լ եմ',
 'Пожалуйста (ответ)',
 'Խնդրեմ',
 'Не стоит благодарности',
 'Հոգ չէ',
 'Не за что',
 'չարժե',
 'Извини!',
 'Ներողությո՜ւն',
 'Извините!',
 'Ներեցե՜ք',
 'извинять',
 'ներել',